In [1]:
tappy_data_path='/home1/vulli/physionet.org/files/tappy/1.0.0/data/Tappy Data/'
users_data_path='/home1/vulli/physionet.org/files/tappy/1.0.0/users/Archived users/'

In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

2023-03-29 04:09:58.327751: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-29 04:09:59.961516: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 11488792683641730698
xla_global_id: -1
]


2023-03-29 04:10:04.670916: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [3]:
import re
import glob
import os
import random
import sys
from tensorflow import keras
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import OneHotEncoder

In [4]:

nb_data_per_person = np.array([0])
def get_user_data():
    import os
    directory = users_data_path
    data = []
    for filename in os.listdir(directory):
        if filename.endswith('.txt'):
            with open(os.path.join(directory, filename)) as f:
                file_contents = f.read()
            file_data = {}
            for line in file_contents.split('\n'):
                if line.strip() != '':
                    key, value = line.split(': ')
                    file_data[key.strip()] = value.strip()

            file_data['ID'] = filename.split("_")[1].split(".")[0]
            # Append the file data to the list
            data.append(file_data)
    df = pd.DataFrame(data)
    df = df[["ID", "Parkinsons"]]
    df["Parkinsons"] = df["Parkinsons"].replace({"False": 0, "True": 1})
    parkinsons_dict = df.set_index('ID')['Parkinsons'].to_dict()
    return parkinsons_dict


In [5]:
fils = sorted(glob.glob(os.path.join(tappy_data_path, '*txt')))
random.shuffle(fils)
parkinsons_dict = get_user_data()
files = []
for file in fils:
    f = file.split("/")[9].split("_")[0]
    if f in parkinsons_dict:
        files.append(file)


In [6]:
def gen_data(temp, had_parkinson_user, data_list):
    count = 0
    print("gen", temp.shape)
    X_data = np.array([])
    Y_data = np.array([])
    nb_datas = int(temp.shape[0] - 100)
    for start in range(0, nb_datas, 80):
        end = start + 100
        data = temp[start:end, :]
        if X_data.size == 0:
            X_data = np.expand_dims(data, axis=-1)
            Y_data = had_parkinson_user
        else:
            X_data = np.dstack((X_data, data))
            Y_data = np.dstack((Y_data, had_parkinson_user))
        count = count + 1
    data_list = np.append(data_list, count+ data_list[-1])
    print(X_data.shape)
    return X_data, Y_data, data_list
    


In [7]:
def one_hot(df, value, cat):
    one_hot_df = pd.get_dummies(df[value], prefix=value, columns=cat)
    df = pd.concat([df, one_hot_df], axis=1)
    df.drop(value, axis=1, inplace=True)
    return df
    

In [8]:
X = np.array([])  # np.ones((self.nb_gait_cycle,self.nb_features))
Y = np.array([])
maximum_rec = 0
for i in range(0, len(files)):
    #print(files[i])
    print("i:", i)
    try:
      data = pd.read_csv(files[i], sep = "\t").iloc[:, :-1]
    except:
      continue
    data.drop(data.columns[[1, 2]], axis=1, inplace=True)
    # print(data.iloc[:0])
    try:
      had_parkinson_user = parkinsons_dict[data.iloc[:, 0].unique()[0]]
    except:
      continue
    data.drop(data.columns[[0]], axis=1, inplace=True) 
    data.columns = ["hand", "hold_time", "dir", "latency_time", "flight_time"]
    data = data[np.in1d(data['hand'], ["L", "R", "S"])
                   & data['hold_time'].apply(lambda x: re.search(r"[^\d.]", str(x)) is None)
                   & np.in1d(data['dir'], ['LL', 'LR', 'RL', 'RR', 'LS', 'SL', 'RS', 'SR', 'SS'])
                   & data['latency_time'].apply(lambda x: re.search(r"[^\d.]", str(x)) is None)
                   & data['flight_time'].apply(lambda x: re.search(r"[^\d.]", str(x)) is None)]
    categories_dir = ['LL', 'LR', 'RL', 'RR', 'LS', 'SL', 'RS', 'SR', 'SS']
    categories_hand = ['L','R','S']
    data = one_hot(data, "hand", categories_hand)
    data = one_hot(data, "dir", categories_dir)
    data = data.reindex(columns=['hold_time', 'latency_time', 'flight_time', 'hand_L', 'hand_R',
       'hand_S', 'dir_LL', 'dir_LR', 'dir_LS', 'dir_RL', 'dir_RR', 'dir_RS',
       'dir_SL', 'dir_SR', 'dir_SS'], fill_value = 0)
    
    #print(data.columns)
    data = data.to_numpy().astype('float64')
    # print(data.shape)
    maximum_rec = max(maximum_rec, data.shape[0])
    if data.shape[0] < 100:
      data = np.pad(data, ((0, 110 - data.shape[0]), (0,0)))
    print(data.shape)
    X_data, y_data , nb_data_per_person = gen_data(data, had_parkinson_user, nb_data_per_person)
    print("X_data", X_data.shape)
    # print((X).size)
    if (X).size == 0:
        X = X_data
        Y = y_data
    else:
        X = np.dstack((X, X_data))
        Y = np.dstack((Y, y_data))
    #print(Y)
    print(X.shape)
           


i: 0
(39360, 15)
gen (39360, 15)
(100, 15, 491)
X_data (100, 15, 491)
(100, 15, 491)
i: 1
(84658, 15)
gen (84658, 15)
(100, 15, 1057)
X_data (100, 15, 1057)
(100, 15, 1548)
i: 2
(110, 15)
gen (110, 15)
(100, 15, 1)
X_data (100, 15, 1)
(100, 15, 1549)
i: 3
(9410, 15)
gen (9410, 15)
(100, 15, 117)
X_data (100, 15, 117)
(100, 15, 1666)
i: 4
(855, 15)
gen (855, 15)
(100, 15, 10)
X_data (100, 15, 10)
(100, 15, 1676)
i: 5
(110, 15)
gen (110, 15)
(100, 15, 1)
X_data (100, 15, 1)
(100, 15, 1677)
i: 6
(2825, 15)
gen (2825, 15)
(100, 15, 35)
X_data (100, 15, 35)
(100, 15, 1712)
i: 7
(6863, 15)
gen (6863, 15)
(100, 15, 85)
X_data (100, 15, 85)
(100, 15, 1797)
i: 8
(524, 15)
gen (524, 15)
(100, 15, 6)
X_data (100, 15, 6)
(100, 15, 1803)
i: 9
(37048, 15)
gen (37048, 15)
(100, 15, 462)
X_data (100, 15, 462)
(100, 15, 2265)
i: 10
(32048, 15)
gen (32048, 15)
(100, 15, 400)
X_data (100, 15, 400)
(100, 15, 2665)
i: 11
(545, 15)
gen (545, 15)
(100, 15, 6)
X_data (100, 15, 6)
(100, 15, 2671)
i: 12
(1818, 

/tmp/SLURM_14209570/ipykernel_31214/1522375010.py:8: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(files[i], sep = "\t").iloc[:, :-1]


(106386, 15)
gen (106386, 15)
(100, 15, 1329)
X_data (100, 15, 1329)
(100, 15, 6291)
i: 24
(394, 15)
gen (394, 15)
(100, 15, 4)
X_data (100, 15, 4)
(100, 15, 6295)
i: 25
(9776, 15)
gen (9776, 15)
(100, 15, 121)
X_data (100, 15, 121)
(100, 15, 6416)
i: 26
(863, 15)
gen (863, 15)
(100, 15, 10)
X_data (100, 15, 10)
(100, 15, 6426)
i: 27
(1064, 15)
gen (1064, 15)
(100, 15, 13)
X_data (100, 15, 13)
(100, 15, 6439)
i: 28
(2085, 15)
gen (2085, 15)
(100, 15, 25)
X_data (100, 15, 25)
(100, 15, 6464)
i: 29
(7275, 15)
gen (7275, 15)
(100, 15, 90)
X_data (100, 15, 90)
(100, 15, 6554)
i: 30
(677, 15)
gen (677, 15)
(100, 15, 8)
X_data (100, 15, 8)
(100, 15, 6562)
i: 31
(166, 15)
gen (166, 15)
(100, 15, 1)
X_data (100, 15, 1)
(100, 15, 6563)
i: 32
(6546, 15)
gen (6546, 15)
(100, 15, 81)
X_data (100, 15, 81)
(100, 15, 6644)
i: 33
(10786, 15)
gen (10786, 15)
(100, 15, 134)
X_data (100, 15, 134)
(100, 15, 6778)
i: 34
(8619, 15)
gen (8619, 15)
(100, 15, 107)
X_data (100, 15, 107)
(100, 15, 6885)
i: 35
(7

/tmp/SLURM_14209570/ipykernel_31214/1522375010.py:8: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(files[i], sep = "\t").iloc[:, :-1]


(243837, 15)
gen (243837, 15)
(100, 15, 3047)
X_data (100, 15, 3047)
(100, 15, 10893)
i: 40
(2222, 15)
gen (2222, 15)
(100, 15, 27)
X_data (100, 15, 27)
(100, 15, 10920)
i: 41
(110, 15)
gen (110, 15)
(100, 15, 1)
X_data (100, 15, 1)
(100, 15, 10921)
i: 42
(110, 15)
gen (110, 15)
(100, 15, 1)
X_data (100, 15, 1)
(100, 15, 10922)
i: 43
(2243, 15)
gen (2243, 15)
(100, 15, 27)
X_data (100, 15, 27)
(100, 15, 10949)
i: 44
(8924, 15)
gen (8924, 15)
(100, 15, 111)
X_data (100, 15, 111)
(100, 15, 11060)
i: 45
i: 46
(1728, 15)
gen (1728, 15)
(100, 15, 21)
X_data (100, 15, 21)
(100, 15, 11081)
i: 47
(11762, 15)
gen (11762, 15)
(100, 15, 146)
X_data (100, 15, 146)
(100, 15, 11227)
i: 48
(216, 15)
gen (216, 15)
(100, 15, 2)
X_data (100, 15, 2)
(100, 15, 11229)
i: 49
(110, 15)
gen (110, 15)
(100, 15, 1)
X_data (100, 15, 1)
(100, 15, 11230)
i: 50
(2163, 15)
gen (2163, 15)
(100, 15, 26)
X_data (100, 15, 26)
(100, 15, 11256)
i: 51
(2398, 15)
gen (2398, 15)
(100, 15, 29)
X_data (100, 15, 29)
(100, 15, 1

...

(122634, 15)
gen (122634, 15)
(100, 15, 1532)
X_data (100, 15, 1532)
(100, 15, 100778)
i: 488
(17512, 15)
gen (17512, 15)
(100, 15, 218)
X_data (100, 15, 218)
(100, 15, 100996)
i: 489
(8370, 15)
gen (8370, 15)
(100, 15, 104)
X_data (100, 15, 104)
(100, 15, 101100)
i: 490
(772, 15)
gen (772, 15)
(100, 15, 9)
X_data (100, 15, 9)
(100, 15, 101109)
i: 491
(4575, 15)
gen (4575, 15)
(100, 15, 56)
X_data (100, 15, 56)
(100, 15, 101165)
i: 492
(166, 15)
gen (166, 15)
(100, 15, 1)
X_data (100, 15, 1)
(100, 15, 101166)
i: 493
(272, 15)
gen (272, 15)
(100, 15, 3)
X_data (100, 15, 3)
(100, 15, 101169)
i: 494
(8271, 15)
gen (8271, 15)
(100, 15, 103)
X_data (100, 15, 103)
(100, 15, 101272)
i: 495
(20103, 15)
gen (20103, 15)
(100, 15, 251)
X_data (100, 15, 251)
(100, 15, 101523)
i: 496
(19927, 15)
gen (19927, 15)
(100, 15, 248)
X_data (100, 15, 248)
(100, 15, 101771)
i: 497
(7447, 15)
gen (7447, 15)
(100, 15, 92)
X_data (100, 15, 92)
(100, 15, 101863)
i: 498
(6636, 15)
gen (6636, 15)
(100, 15, 82)
X_

/tmp/SLURM_14209570/ipykernel_31214/1522375010.py:8: DtypeWarning: Columns (6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(files[i], sep = "\t").iloc[:, :-1]


(73488, 15)
gen (73488, 15)
(100, 15, 918)
X_data (100, 15, 918)
(100, 15, 108978)
i: 536


/tmp/SLURM_14209570/ipykernel_31214/1522375010.py:8: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(files[i], sep = "\t").iloc[:, :-1]


(71100, 15)
gen (71100, 15)
(100, 15, 888)
X_data (100, 15, 888)
(100, 15, 109866)
i: 537
(8318, 15)
gen (8318, 15)
(100, 15, 103)
X_data (100, 15, 103)
(100, 15, 109969)
i: 538
(12107, 15)
gen (12107, 15)
(100, 15, 151)
X_data (100, 15, 151)
(100, 15, 110120)
i: 539
(1214, 15)
gen (1214, 15)
(100, 15, 14)
X_data (100, 15, 14)
(100, 15, 110134)
i: 540
(5144, 15)
gen (5144, 15)
(100, 15, 64)
X_data (100, 15, 64)
(100, 15, 110198)
i: 541
(418, 15)
gen (418, 15)
(100, 15, 4)
X_data (100, 15, 4)
(100, 15, 110202)
i: 542
(20264, 15)
gen (20264, 15)
(100, 15, 253)
X_data (100, 15, 253)
(100, 15, 110455)
i: 543
(5536, 15)
gen (5536, 15)
(100, 15, 68)
X_data (100, 15, 68)
(100, 15, 110523)
i: 544
(8310, 15)
gen (8310, 15)
(100, 15, 103)
X_data (100, 15, 103)
(100, 15, 110626)
i: 545
(555, 15)
gen (555, 15)
(100, 15, 6)
X_data (100, 15, 6)
(100, 15, 110632)
i: 546
(21692, 15)
gen (21692, 15)
(100, 15, 270)
X_data (100, 15, 270)
(100, 15, 110902)
i: 547
(935, 15)
gen (935, 15)
(100, 15, 11)
X_da

In [9]:
X = X.transpose(2, 0 , 1)
Y = Y.transpose(2, 0, 1)

In [10]:
X.shape

(112307, 100, 15)

In [11]:
X1_copy = np.copy(X)
Y1_copy = np.copy(Y)

In [12]:
X1_copy.shape

(112307, 100, 15)

In [13]:
from sklearn.model_selection import train_test_split
train_size=0.8

# In the first step we will split the data in training and remaining dataset
X_train, X_rem, y_train, y_rem = train_test_split(X,Y, train_size=0.8)

# Now since we want the valid and test size to be equal (10% each of overall data). 
# we have to define valid_size=0.5 (that is 50% of remaining data)
test_size = 0.5
X_valid, X_test, y_valid, y_test = train_test_split(X_rem,y_rem, test_size=0.5)

print(X_train.shape), print(y_train.shape)
print(X_valid.shape), print(y_valid.shape)
print(X_test.shape), print(y_test.shape)

(89845, 100, 15)
(89845, 1, 1)
(11231, 100, 15)
(11231, 1, 1)
(11231, 100, 15)
(11231, 1, 1)


(None, None)

In [15]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, CuDNNLSTM, Normalization,Dropout

# Create the model
model = Sequential()

model.add(tf.keras.layers.Masking(mask_value=0.,
                                  input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Normalization(axis=(1, 2)))
model.add(LSTM(128, input_shape=(X_train.shape[1], X_train.shape[2]),return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(128, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
# model.add(Softmax())
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Fit the model to the data
model.fit(X_train, y_train, epochs=20, batch_size=32)

2023-03-29 04:18:46.147143: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-03-29 04:18:46.148890: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-03-29 04:18:46.150345: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Epoch 1/20


2023-03-29 04:18:47.657583: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-03-29 04:18:47.659431: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-03-29 04:18:47.661036: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

 132/2808 [>.............................] - ETA: 10:06 - loss: 0.5768 - accuracy: 0.7145

KeyboardInterrupt: 

In [ ]:
# Evaluate the model on new data
# new_X =  X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))
model.evaluate(X_valid, y_valid)

In [ ]:
model.summary()

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import Input, LSTM, Dense, Concatenate, TimeDistributed

# Define the LSTM layer with desired parameters
lstm_units = 64
lstm_layer = LSTM(units=lstm_units, activation='tanh', recurrent_activation='sigmoid', return_sequences=True)

# Create the input layer for the LSTM layer
input_lstm = Input(shape=(100, 5))

# Use TimeDistributed layer to apply the LSTM layer to each time step in the input data
lstm_output = TimeDistributed(lstm_layer)(input_lstm)

# Define the dense layer with desired parameters
dense_units = 32
dense_layer = Dense(units=dense_units, activation='relu')

# Create a concatenation layer to combine the output of the LSTM layer and the second data
input_dense = Input(shape=(5,))
concatenated_data = Concatenate()([lstm_output, input_dense])

# Define the output layer with desired parameters
output_units = 1
output_layer = Dense(units=output_units, activation='sigmoid')

# Build the model
model = keras.models.Model(inputs=[input_lstm, input_dense], outputs=output_layer(concatenated_data))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(x=[data_1, data_2], y=labels, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate the model
loss, accuracy = model.evaluate(x=[test_data_1, test_data_2], y=test_labels)
print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")
